In [2]:
from canvasapi import Canvas, assignment
import canvasapi as cnv
from tokens import canvas_token, notion_token
from notion_client import Client
from datetime import datetime,timedelta
import pytz

In [3]:
nclient = Client(auth=notion_token)
API_URL = "https://osu.instructure.com/"
API_KEY = canvas_token
canvas = Canvas(API_URL,API_KEY)
kiran = canvas.get_current_user()
courses = []
enrollment_term_id = 164
for course in kiran.get_courses():
    if course.__dict__['enrollment_term_id'] == enrollment_term_id:
        courses.append(course)
db_id = "57c6c1fdb5b54861af2fb7787344b4fc"
subjects = {}

In [19]:
p1 = nclient.pages.create()

APIResponseError: body failed validation: body.parent should be defined, instead was `undefined`.

In [4]:
def get_schema(assignment:assignment,subject_dict:dict,course:str):
    status = ""
    if assignment.due_at_date.date()-datetime.today().date()<=timedelta(days=7):
        status = str(1)
    else:
        status = str(2)
    if course in subject_dict.keys():
        subject = subject_dict[course]
    else:
        subject = {'name':course}
    schema = {
    "Status":{"status":{"id":status}},
    'Name': {"title": [{"type":"text","text":{"content":assignment.name}}]},
    "Subject":{"select":subject},
    "Due Date":{"date":{"start":datetime.strptime(assignment.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}},
    "Link":{"url":assignment.html_url},
    "Time to complete":{"select":{'id': 'b261419e-254f-44b9-8788-1532a54646fb','name': 'About an hour','color': 'red'}}
    }
    return schema

In [6]:
def add_assignment(assignment:assignment,subject_dict:dict,course:str):
    return nclient.pages.create(**{"parent":{"type":"database_id","database_id":db_id},"properties":get_schema(assignment,subjects,course)})


In [7]:
def update_course(course:cnv.course.Course,subject_dict:dict):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if course.name in subject_dict.keys():
            subject = subject_dict[course.name]['name']
        else:
            subject = course.name
        if len(nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results'])<1 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
            print(x)
            print(add_assignment(assignment=x,subject_dict=subject_dict,course=course.name)['properties']['Name'])

In [16]:
def update_all(courses:list,subject_dict:dict):
    backlog = "Backlog"
    week_backlog = "Week Backlog"
    backloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","status":{"equals":"Backlog"}}})['results']
    for page in backloglist:
        date = datetime.fromisoformat(page['properties']['Due Date']['date']['start'])
        if date.date()-datetime.today().date()<timedelta(days=7):
            nclient.pages.update(page['id'],**{'properties':{'Status':{'status':{'name':week_backlog}}}})
    for x in courses:
        update_course(x,subject_dict)

In [ ]:
update_all(courses=courses,subject_dict=subjects)

In [303]:
def add_links(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Name","title":{"contains":x.name}}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Link':{'url':x.html_url}}})

In [57]:
def update_times(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subjects[course.name]['name']}}]}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Due Date':{"date":{"start":datetime.strptime(x.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}}}})

In [40]:
def find_duplicates(course_list):
    for c in course_list:
        course_assignments = [y for y in kiran.get_assignments(c)]
        for x in course_assignments:
            subject = c.name
            if len(nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results'])>0 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
                print(x)

In [53]:
def remove_duplicates(course_list):
    for c in course_list:
        course_assignments = [y for y in kiran.get_assignments(c)]
        for x in course_assignments:
            subject = c.name
            qresults = nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results']
            if len(qresults)>1 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
                for x in range(len(qresults)-1):
                    currentpage = nclient.pages.update(qresults[x]['id'],**{'archived':True})